<center><h1>Demo cách làm vector Doc2vec ko làm luôn vì máy cùi</h1></center>

In [231]:
from gensim.models import Word2Vec
import os
import warnings
from pyvi import ViTokenizer
import re
warnings.filterwarnings("ignore")
# Load the data
path = os.getcwd() + '/../Data/demo-full.txt'
sentences = open(path, 'r', encoding='utf-8').read()
stop_words = open(os.getcwd() + '/../Data/vietnamese-stopwords.txt', 'r', encoding='utf-8').read()
stop_words = stop_words.split()
# sentences[:100]
# with open(path, 'r', encoding='utf-8') as f:
#     sentences = f.readlines()

In [232]:
def sentences_processing(sentences):
    sentences = sentences.lower()
    sentences = ViTokenizer.tokenize(sentences)
    sentences = sentences.split('\n')
    # remove all the special characters
    sentences = [re.sub(r'\W', ' ', sentence) for sentence in sentences]
    # remove all single characters
    sentences = [re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence) for sentence in sentences]
    # remove single characters from the start
    sentences = [re.sub(r'\^[a-zA-Z]\s+', ' ', sentence) for sentence in sentences]
    # remove numbers
    sentences = [re.sub(r'\d+', '', sentence) for sentence in sentences]
    #  remove stop words
    sentences = [[word for word in sentence.split() if word not in stop_words] for sentence in sentences]
    #  remove words with length < 3
    sentences = [[word for word in sentence if len(word) > 2] for sentence in sentences]
    #  remove words with length > 15
    sentences = [[word for word in sentence if len(word) < 15] for sentence in sentences]
    #  remove empty sentences
    sentences = [sentence for sentence in sentences if len(sentence) > 0]
    return sentences
sentences = sentences_processing(sentences)



In [233]:
#  Train the model
skip_gram = Word2Vec(sentences, min_count=1, vector_size=100, window=3, sg=1)
#  get the sentence vector
def get_sentence_vector(sentence):
    sentence = sentence.lower()
    sentence = ViTokenizer.tokenize(sentence)
    sentence = re.sub(r'\W', ' ', sentence)
    sentence = [word for word in sentence.split() if word not in stop_words]
    sentence = [word for word in sentence if len(word) > 2]
    sentence = [word for word in sentence if len(word) < 15]
    sentence = [word for word in sentence if word in skip_gram.wv.vocab]
    if len(sentence) == 0:
        return None
    return sum([skip_gram.wv[word] for word in sentence]) / len(sentence)

#  get the similarity between two sentences
def get_similarity(sentence1, sentence2):
    sentence1 = get_sentence_vector(sentence1)
    sentence2 = get_sentence_vector(sentence2)
    if sentence1 is None or sentence2 is None:
        return 0
    return skip_gram.wv.cosine_similarities(sentence1, sentence2)[0]

#  get the similarity matrix
def get_similarity_matrix(sentences):
    n = len(sentences)
    similarity_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            similarity_matrix[i][j] = get_similarity(sentences[i], sentences[j])
    return similarity_matrix

g


In [234]:
#  Save the model
skip_gram.save('skip_gram.bin')
#  Load the model
skip_gram = Word2Vec.load('skip_gram.bin')
#  get the 
skip_gram.wv['thủ_tướng']


In [235]:
skip_gram.wv.most_similar(positive=['tiền', 'thuế'], negative=['lương'])

[('đường', 0.9859349131584167),
 ('thu', 0.9857032299041748),
 ('thọ', 0.9855136871337891),
 ('thông_tin', 0.9853044748306274),
 ('xử_phạt', 0.9852707982063293),
 ('khu', 0.9851948022842407),
 ('quỹ', 0.9851796627044678),
 ('vi_phạm', 0.9851405024528503),
 ('hai', 0.984961211681366),
 ('trả', 0.9849055409431458)]